In [22]:
println("Loading libraries and data..")
using AlgebraicSolving
#import Nemo:
#    is_univariate,
#    coefficients_of_univariate
using Nemo
#using Oscar
#include("tools.jl")

Loading libraries and data..


In [23]:

include("data.jl")
include("src/usolve/usolve.jl")
#using Arblib

usolve (generic function with 1 method)

In [130]:
function order_permut2d(L)
    # Create a list of tuples with elements and their corresponding indices
    LL = [(L[i][j], (i, j)) for i in eachindex(L) for j in eachindex(L[i])]
    # Sort the enumerated list based on the values
    sorted_LL = sort(LL, by = x -> x[1])
    # Extract the sorted values and their corresponding indices
    sorted_L = [pair[1] for pair in sorted_LL]
    sorted_ind = [pair[2] for pair in sorted_LL]
    
    return sorted_L, sorted_ind
end

function isolate(f; prec = 32, software="usolve")
    @assert is_univariate(f) "Not univariate polynomial"
	if software == "usolve"
		return usolve(f, precision = prec, uspath="src/usolve/usolve")
    end
end

function isolate_eval(f, ivar, val; prec=64)
    sols = real_solutions(AlgebraicSolving.Ideal([f, gens(R)[ivar] - val]), precision=prec)
    return [ s[ivar%2+1] for s in sols ]
end

function Arb_to_rat(x)
	r = radius(x)
	return map(simplest_rational_inside, [x-2*r, x+2*r])
end

function rat_to_Arb(x)
    x1,x2 = x
    xm, xd = RR((x1+x2)/2), RR(x2-x1)
    return ball(xm,xd)
end

function evaluate_Arb(f, x)
	cf = coefficients_of_univariate(f)
	return evalpoly(RR(x), cf) 
end

evaluate_Arb (generic function with 1 method)

In [131]:
println("Isolating critical values..")
xcrit = [ isolate(first(p), prec=150) for p in params ]
xcrit_usolve = getindex.(xcrit, 1)
xcrit = getindex.(xcrit, 2)
xcritorder, xcritpermut = order_permut2d(xcrit);

Isolating critical values..


In [132]:
println("Computing isolating critical boxes using Arb..")
#RR(x) = Arb(x, prec=120)
RR = ArbField(150)
Pcrit = [ [ [xc, evaluate_Arb(params[i][2], xc[1])/evaluate_Arb(params[i][3],xc[1])] for xc in xcrit[i]] for i in eachindex(xcrit) ]
LBcrit = [ [ [ map(QQ, pc[1]), map(QQ, Arb_to_rat(pc[2])) ]  for pc in pcrit] for pcrit in Pcrit ];
Pcrit = [ [ [ rat_to_Arb(pc[1]), pc[2] ]  for pc in pcrit] for pcrit in Pcrit ];

Computing isolating critical boxes using Arb..


In [187]:
Pcrit[1][1][2]

[0.00592297584970014544321252 +/- 4.43e-27]

In [191]:
# To try/do : isolate with usolve, call msolve with only one variable
function intersect_box(f, B)
    L = Array{Any}(undef, 4)
    for i in 1:2
        # Lxi
        L[i] = Array{Any}(undef,2)
        L[i][1] = isolate_eval(f, 2, B[2][i], prec=150)
        L[i][2] = [ j for (j, l) in pairs(L[i][1]) if (l> B[1][1] && l<B[1][2]) ]
        # Lyi
        L[i+2] = Array{Any}(undef,2)
        L[i+2][1] = isolate_eval(f, 1, B[1][i], prec=150)
        L[i+2][2] =  [ j for (j, l) in pairs(L[i+2][1]) if (l> B[2][1] && l<B[2][2]) ]
    end
    return L
end

intersect_box (generic function with 1 method)

In [192]:
function refine_xboxes(f, LBcrit)
    # Implementation of refine_xboxes function goes here
end

refine_xboxes (generic function with 1 method)

In [193]:
println("\nCompute intersections with critical boxes")
ts = time()
LPCside = Array{Any}(undef,2)
ndig = maximum([Int(floor(log10(length(LBcrit[i])))) for i in 1:2])
Ltm = []
i = 1
while i <= length(LBcrit)
    ndigi = Int(floor(log10(length(LBcrit[i]))))
    LPCside[i] = Array{Any}(undef, length(LBcrit[i]))
    for j in eachindex(LBcrit[i])
        print("mult=$i ; $(j)/$(length(LBcrit[i]))$(repeat(" ", ndig-ndigi+1))pts","\r")
        tm = time()
        pcside = intersect_box(f, LBcrit[i][j])
        npcside = [length(n) for (I, n) in pcside]
        if i == 1 && sum(npcside) > 2
            @assert false "\nRefine extreme boxes along x-axis"
            #refine_xboxes(QQ["x"](params[1][1]), LBcrit[1])
            #i = 0
            #j = 1
            #break
        elseif i > 1 && sum(npcside[1:2]) != 0
            @assert false "\nRefine singular boxes along x-axis"
            #refine_xboxes(QQ["x"](params[2][1]), LBcrit[2])
            #i -= 1
            #break
        end
        LPCside[i][j] = pcside
        push!(Ltm, time() - tm)
    end
    i += 1
    println("")
end
LnPCside = [ [[length(indI) for (L, indI) in PB] for PB in lpcside] for lpcside in LPCside ] 
println("Average time for one point: $(mean(Ltm))s")
println("Elapsed time to compute critical boxes: $(time()-ts)s")



Compute intersections with critical boxes
mult=1 ; 16/16 pts
mult=2 ; 60/60 pts
Average time for one point: 0.25893976186451156s
Elapsed time to compute critical boxes: 19.82736611366272s


In [190]:
y1, y2 = [ l[1] for l in LPCside[1][1] ][4]
yB1, yB2 = LBcrit[1][1][2]
yRB = Pcrit[1][1][2]
print(y2<yB2)

true

In [196]:
# Update extreme boxes
for j in eachindex(LBcrit[1])
    # If the curve does not intersect the box only on vertical sides or not at all
    if !(LnPCside[1][j][1:2] == [0, 0]) || sum(LnPCside[1][j])==0
        PCside, nPCside = LPCside[1][j], LnPCside[1][j]
        I = [ l[1] for l in PCside[3:end] ]
        nI = [ l[2] for l in PCside[3:end] ]
        # Locate the orientation of the extreme point
        # s is the index on the side where there are more branches
        # s=1: left; s=2: right
        s = argmax([length(I[1]), length(I[2])])
        # Ordinate range of the extreme point
        ycrit = Pcrit[1][j][2]
        # If it intersects on the bottom side
        if nPCside[1] == 1
            # yinf: the intersection with the vertical side just below the extreme point
            yinf = maximum([i for (i, yy) in pairs(I[s]) if yy < ycrit])
            # We vertically enlarge the box until it intersects on the horizontal side
            push!(LPCside[1][j][s + 2][2], yinf)
            LPCside[1][j][1][2] = []
            # We update the intersection numbers
            LnPCside[1][j][s + 2] += 1
            LnPCside[1][j][1] = 0
        end
        # If it intersects on the top side
        if nPCside[2] == 1 
            # ymax: the intersection with the vertical side just above the extreme point
            ymax = minimum([i for (i, yy) in pairs(I[s]) if yy > ycrit])
            # We vertically enlarge the box until it intersects on the horizontal side
            push!(LPCside[1][j][s + 2][2], ymax)
            LPCside[1][j][2][2] = []
            # We update the intersection numbers
            LnPCside[1][j][s + 2] += 1
            LnPCside[1][j][2] = 0
        end
    end
end

In [205]:
print("Graph computation")
# Would be nice to have only one intermediate fiber (take the average of abscissa and ordinates) for plot
# And even remove this fiber for the graph
Vert = []
Edg = []
Corr = [[[[], [[], [], []], []] for j in xcrit[i] ] for i in eachindex(xcrit) ]
Viso = []
ite = 1

for ind in 1:length(xcritpermut)
    i, j = xcritpermut[ind]
    if ind > 1
        i1, j1 = xcritpermut[ind - 1]
    end
    if ind < length(xcritpermut)
        i2, j2 = xcritpermut[ind + 1]
        I2L, nI2L = LPCside[i2][j2][2]
    end

    PCside, nPCside = LPCside[i][j], LnPCside[i][j]
    I = [ l[1] for l in PCside[3:end] ]
    nI = [ l[2] for l in PCside[3:end] ]
    
    ymincrit = min(nI[1][1] + nI[1][2] + [length(I[1][1])])

    # Construct vertices
    ###########################
    # On the vertical left side
    if ind > 1
        for k in 1:length(I[1][1])
            push!(Corr[i][j][1][1], Corr[i1][j1][3][k])
        end
    else
        for k in 1:length(I[1][1])
            push!(Vert, (xcrit[i][j].lower() - exp, I[1][1][k]))
            push!(Corr[i][j][1][1], ite)
            ite += 1
        end
    end
    ###########################
    # On the vertical right side
    if ind < length(xcritpermut)
        for k in 1:length(I[1][2])
            push!(Vert, ((xcrit[i][j] + xcrit[i2][j2]).lower() / 2 + exp, (I[1][2][k] + I2L[k]).lower() / 2))
            push!(Corr[i][j][3], ite)
            ite += 1
        end
    else
        for k in 1:length(I[1][2])
            push!(Vert, (xcrit[i][j].upper() + exp, I[1][2][k]))
            push!(Corr[i][j][3], ite)
            ite += 1
        end
    end
    ###########################
    # Below the critical point
    for k in 1:ymincrit
        push!(Vert, (xcrit[i][j], (I[1][1][k] + I[1][2][k]) / 2))
        push!(Corr[i][j][2][1], ite)
        push!(Edg, (Corr[i][j][1][1][k], ite))  # left
        push!(Edg, (ite, Corr[i][j][3][k]))  # right
        ite += 1
    end
    ###########################
    # The critical point
    ##########################
    # if it is an isolated point
    if isempty(nI[1][1]) && isempty(nI[1][2])
        #pass
        # We can add the isolated  vertex
        # push!(Vert, Pcrit[i][j])
        # push!(Corr[i][j][2][1], ite)
        # We will subsequently add the vertex in the graph
        # push!(Viso, ite)
        # ite += 1
    ############################################
    ## TO BE REPLACED BY APPSING IDENTIFICATOR ##
    ## works for space curves without nodes   ##
    ############################################
    # If we are dealing with a node
    elseif i == 1
        # We connect the pairwise opposite branches nI[1][1][i] and nI[1][2][i+1 mod 2], i=1,2
        push!(Edg, (Corr[i][j][1][1][nI[1][1][1]], Corr[i][j][3][nI[1][2][2]]))
        push!(Edg, (Corr[i][j][1][1][nI[1][1][2]], Corr[i][j][3][nI[1][2][1]]))
    else
        # We can add the vertex
        push!(Vert, Pcrit[i][j])
        push!(Corr[i][j][2][1], ite)
        # We connect to the vertical left side of the critical box
        for k in nI[1][1]
            push!(Edg, (Corr[i][j][1][1][k], ite))
        end
        # We connect to the vertical right side of the critical box
        for k in nI[1][2]
            push!(Edg, (ite, Corr[i][j][3][k]))
        end
        ite += 1
    end
    ###########################
    # Above the critical point
    for k in length(I[1][1]) - length(nI[1][1]) - ymincrit:-1:1
        push!(Vert, (xcrit[i][j], (I[1][1][end - k + 1] + I[1][2][end - k + 1]) / 2))
        push!(Corr[i][j][2][3], ite)
        push!(Edg, (Corr[i][j][1][1][end - k + 1], ite))  # left
        push!(Edg, (ite, Corr[i][j][3][end - k + 1]))  # right
        ite += 1
    end
end

Graph computation
I :Vector{QQFieldElem}[[], []]

BoundsError: BoundsError: attempt to access 0-element Vector{QQFieldElem} at index [1]

UndefVarError: UndefVarError: `LnPCside` not defined

In [21]:
I